# Part 0: Download LD panel

In [4]:
using EasyLD
using CSV
using DataFrames
using Statistics
using LinearAlgebra

Check how many files are there

In [2]:
@show length(get_ukb_filenames("EUR", join=false))
@show length(get_ukb_filenames("AFR", join=false))
@show length(get_ukb_filenames("EAS", join=false));

length(get_ukb_filenames("EUR", join = false)) = 124281
length(get_ukb_filenames("AFR", join = false)) = 106258
length(get_ukb_filenames("EAS", join = false)) = 24892


Try downloading the first 10 files (~1GB) for EUR

In [3]:
population = "EUR"
outdir = "/scratch/users/bbchu/LD_matrices"
download_ukb_LD_matrices(population, outdir, start_from=1, num_files=10)

Progress: 100%|█████████████████████████████████████████| Time: 0:02:47


Also download variant index hail table

In [7]:
population = "EUR"
outdir = "/oak/stanford/groups/zihuai/pan_ukb_LD_matrices"
download_ukb_variant_index_tables(population, outdir)

download: s3://pan-ukb-us-east-1/ld_release/UKBB.EUR.ldadj.variant.ht/_SUCCESS to ../../../../scratch/users/bbchu/LD_matrices/UKBB.EUR.ldadj.variant.ht/_SUCCESS
download: s3://pan-ukb-us-east-1/ld_release/UKBB.EUR.ldadj.variant.ht/globals/parts/part-0 to ../../../../scratch/users/bbchu/LD_matrices/UKBB.EUR.ldadj.variant.ht/globals/parts/part-0
download: s3://pan-ukb-us-east-1/ld_release/UKBB.EUR.ldadj.variant.ht/index/part-00002-38-2-0-864c49d5-6ecf-d3d1-7a84-3208d8861c9f.idx/metadata.json.gz to ../../../../scratch/users/bbchu/LD_matrices/UKBB.EUR.ldadj.variant.ht/index/part-00002-38-2-0-864c49d5-6ecf-d3d1-7a84-3208d8861c9f.idx/metadata.json.gz
download: s3://pan-ukb-us-east-1/ld_release/UKBB.EUR.ldadj.variant.ht/index/part-00000-38-0-0-6427c99b-9cd4-7c36-673b-98da4d7d69bf.idx/metadata.json.gz to ../../../../scratch/users/bbchu/LD_matrices/UKBB.EUR.ldadj.variant.ht/index/part-00000-38-0-0-6427c99b-9cd4-7c36-673b-98da4d7d69bf.idx/metadata.json.gz
download: s3://pan-ukb-us-east-1/ld_rele

Excessive output truncated after 524334 bytes.

## Download all files in parallel

In [ ]:
# put in file /oak/stanford/groups/zihuai/pan_ukb_LD_matrices/download.jl

using EasyLD
using CSV
using DataFrames
using Statistics
using LinearAlgebra

# arguments to this script
population = ARGS[1]
batch = parse(Int, ARGS[2])

# run download
batch_size = 1000
start_from = batch_size*(batch-1) + 1
outdir = "/oak/stanford/groups/zihuai/pan_ukb_LD_matrices"
download_ukb_LD_matrices(population, outdir, start_from=start_from, num_files=batch_size)

In [7]:
using CSV, DataFrames

global download_exe = "/oak/stanford/groups/zihuai/pan_ukb_LD_matrices/download.jl"

function submit_download(population::String)
    if population == "EUR"
        nfiles = 124281
    elseif population == "AFR"
        nfiles = 106258
    elseif population == "EAS"
        nfiles = 24892
    else
        error("population should be EUR, AFR, or EAS")
    end
    batchsize = 1000
    batches = div(nfiles, batchsize, RoundUp)
    outdir = "/oak/stanford/groups/zihuai/pan_ukb_LD_matrices"
    for b in 1:batches
        # create .sh file to submit jobs
        filename = "submit.sh"
        open(filename, "w") do io
            println(io, "#!/bin/bash")
            println(io, "#")
            println(io, "#SBATCH --job-name=$population.$b")
            println(io, "#")
            println(io, "#SBATCH --time=24:00:00")
            println(io, "#SBATCH --cpus-per-task=1")
            println(io, "#SBATCH --mem-per-cpu=4G")
            println(io, "#SBATCH --partition=normal,candes,zihuai")
            println(io, "#SBATCH --output=$(outdir)/slurms/slurm-%j.out")
            println(io, "")
            println(io, "#save job info on joblog:")
            println(io, "echo \"Job \$JOB_ID started on:   \" `hostname -s`")
            println(io, "echo \"Job \$JOB_ID started on:   \" `date `")
            println(io, "")
            println(io, "# load the job environment:")
            println(io, "module load openssl/3.0.7")
            println(io, "module load julia/1.8.4")
            println(io, "module load R/4.0.2")
            println(io, "module load java/11.0.11")
            println(io, "module load python/3.9.0")
            println(io, "export JULIA_DEPOT_PATH=\"/home/groups/sabatti/.julia\"")
            println(io, "")
            println(io, "# run code")
            println(io, "echo 'julia $download_exe $population $b'")
            println(io, "julia $download_exe $population $b")
            println(io, "")
            println(io, "#echo job info on joblog:")
            println(io, "echo \"Job \$JOB_ID ended on:   \" `hostname -s`")
            println(io, "echo \"Job \$JOB_ID ended on:   \" `date `")
            println(io, "#echo \" \"")
        end
        # submit job
        run(`sbatch $filename`)
        println("submitted batch $b") 
        rm(filename, force=true)
    end
end

population = "EAS" # EUR, AFR, or EAS
submit_download(population)

Submitted batch job 25893856
submitted batch 1
Submitted batch job 25893857
submitted batch 2
Submitted batch job 25893858
submitted batch 3
Submitted batch job 25893859
submitted batch 4
Submitted batch job 25893860
submitted batch 5
Submitted batch job 25893861
submitted batch 6
Submitted batch job 25893862
submitted batch 7
Submitted batch job 25893863
submitted batch 8
Submitted batch job 25893865
submitted batch 9
Submitted batch job 25893868
submitted batch 10
Submitted batch job 25893869
submitted batch 11
Submitted batch job 25893870
submitted batch 12
Submitted batch job 25893871
submitted batch 13
Submitted batch job 25893872
submitted batch 14
Submitted batch job 25893873
submitted batch 15
Submitted batch job 25893874
submitted batch 16
Submitted batch job 25893875
submitted batch 17
Submitted batch job 25893876
submitted batch 18
Submitted batch job 25893877
submitted batch 19
Submitted batch job 25893878
submitted batch 20
Submitted batch job 25893879
submitted batch 21
S

In [3]:
for population in ["AFR", "EAS"]
    outdir = "/oak/stanford/groups/zihuai/pan_ukb_LD_matrices"
    download_ukb_variant_index_tables(population, outdir)
end

download: s3://pan-ukb-us-east-1/ld_release/UKBB.AFR.ldadj.variant.ht/README.txt to ../../../../oak/stanford/groups/zihuai/pan_ukb_LD_matrices/UKBB.AFR.ldadj.variant.ht/README.txt
download: s3://pan-ukb-us-east-1/ld_release/UKBB.AFR.ldadj.variant.ht/globals/parts/part-0 to ../../../../oak/stanford/groups/zihuai/pan_ukb_LD_matrices/UKBB.AFR.ldadj.variant.ht/globals/parts/part-0
download: s3://pan-ukb-us-east-1/ld_release/UKBB.AFR.ldadj.variant.ht/_SUCCESS to ../../../../oak/stanford/groups/zihuai/pan_ukb_LD_matrices/UKBB.AFR.ldadj.variant.ht/_SUCCESS
download: s3://pan-ukb-us-east-1/ld_release/UKBB.AFR.ldadj.variant.ht/index/part-00000-38-0-0-c97a4f71-06af-5a5f-72d6-96952cd5f2b6.idx/index to ../../../../oak/stanford/groups/zihuai/pan_ukb_LD_matrices/UKBB.AFR.ldadj.variant.ht/index/part-00000-38-0-0-c97a4f71-06af-5a5f-72d6-96952cd5f2b6.idx/index
download: s3://pan-ukb-us-east-1/ld_release/UKBB.AFR.ldadj.variant.ht/globals/metadata.json.gz to ../../../../oak/stanford/groups/zihuai/pan_ukb

Excessive output truncated after 524372 bytes.

download: s3://pan-ukb-us-east-1/ld_release/UKBB.AFR.ldadj.variant.ht/index/part-00530-38-530-0-cddf9132-0ced-b064-6815-48b7be7c9500.idx/index to ../../../../oak/stanford/groups/zihuai/pan_ukb_LD_matrices/UKBB.AFR.ldadj.variant.ht/index/part-00530-38-530-0-cddf9132-0ced-b064-6815-48b7be7c9500.idx/index
download: s3://pan-ukb-us-east-1/ld_release/UKBB.AFR.ldadj.variant.ht/index/part-00531-38-531-0-05db8348-43b1-4f59-0cf7-5d288f4b8b2a.idx/index to ../../../../oak/stanford/groups/zihuai/pan_ukb_LD_matrices/UKBB.AFR.ldadj.variant.ht/index/part-00531-38-531-0-05db8348-43b1-4f59-0cf7-5d288f4b8b2a.idx/index
download: s3://pan-ukb-us-east-1/ld_release/UKBB.AFR.ldadj.variant.ht/index/part-00533-38-533-0-cfc705a9-37d3-d1ea-4544-66fd48505e05.idx/index to ../../../../oak/stanford/groups/zihuai/pan_ukb_LD_matrices/UKBB.AFR.ldadj.variant.ht/index/part-00533-38-533-0-cfc705a9-37d3-d1ea-4544-66fd48505e05.idx/index
download: s3://pan-ukb-us-east-1/ld_release/UKBB.AFR.ldadj.variant.ht/index/part-00532-3

## Try importing AFR panel

The first time this is ran, need to do some preprocessing which requires ~64GB, so these were ran in the terminal

In [ ]:
dir = "/oak/stanford/groups/zihuai/pan_ukb_LD_matrices"
bm_file = joinpath(dir, "UKBB.AFR.ldadj.bm")
ht_file = joinpath(dir, "UKBB.AFR.ldadj.variant.ht")

@time bm = hail_block_matrix(bm_file, ht_file);

In [ ]:
Sigma = bm[1:10000, 1:10000]

## Try importing EAS panel

In [ ]:
dir = "/oak/stanford/groups/zihuai/pan_ukb_LD_matrices"
bm_file = joinpath(dir, "UKBB.EAS.ldadj.bm")
ht_file = joinpath(dir, "UKBB.EAS.ldadj.variant.ht")

@time bm = hail_block_matrix(bm_file, ht_file);

In [ ]:
Sigma = bm[1:10000, 1:10000]